#### https://www.kaggle.com/nehalbirla/vehicle-dataset-from-cardekho/tasks

## Loading Libraries and Dataset 

In [1]:
# LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# IMPORT DATASET THEN SPLIT INDEPENDENT AND DEPENDENT VARIABLE
df = pd.read_csv('Car details v3.csv')

#### Preprocessing 

In [3]:
#Process name, into brand then model
brand = df['name'].str.split(' ').str[0]

In [4]:
#Replace name with brand
brand = df['name'].str.split(' ').str[0]
df['name'] = brand
df.rename(columns ={'name' : 'brand'}, inplace = True)

In [5]:
engine = df['engine'].str.split(' ').str[0]
engine = engine.astype('float')
df['engine'] = engine
df.rename(columns ={'engine' : 'engine(cc)'}, inplace = True)

### Removed row 6730, cannot be used for prediction for specific brand, also has NaN values that cannot be replaced with mean since there are no other cars like this, Dropped Torque, BHP is enough 

In [6]:
#Torque Column removed, Row 6730 removed, due to NaN values to be replaced by groupedby mean
df = df.drop(6730, axis = 0)
df = df.drop(columns = 'torque')
df.reset_index()

,index,brand,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine(cc),max_power,seats
0,0,Maruti,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248.0,74 bhp,5.0
1,1,Skoda,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498.0,103.52 bhp,5.0
2,2,Honda,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497.0,78 bhp,5.0
3,3,Hyundai,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396.0,90 bhp,5.0
4,4,Maruti,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298.0,88.2 bhp,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8122,8123,Hyundai,2013,320000,110000,Petrol,Individual,Manual,First Owner,18.5 kmpl,1197.0,82.85 bhp,5.0
8123,8124,Hyundai,2007,135000,119000,Diesel,Individual,Manual,Fourth & Above Owner,16.8 kmpl,1493.0,110 bhp,5.0
8124,8125,Maruti,2009,382000,120000,Diesel,Individual,Manual,First Owner,19.3 kmpl,1248.0,73.9 bhp,5.0
8125,8126,Tata,2013,290000,25000,Diesel,Individual,Manual,First Owner,23.57 kmpl,1396.0,70 bhp,5.0


In [7]:
#Set all mileage to only kmpl scale | kg * 0.51
mileage = df['mileage'].str.split(' ').str[0]
metric = df['mileage'].str.split(' ').str[1]
kg = metric.index[metric == 'km/kg']
mileage = mileage.astype('float')
mileage[kg] = mileage[kg] * 0.9
df['mileage'] = mileage
df.rename(columns ={'mileage' : 'mileage(kmpl)'}, inplace = True)

In [8]:
#Max_power to max_power(bhp)
max_power = df['max_power'].str.split(' ', expand = True)
max_power = max_power[0]
max_power[max_power == ''] = np.nan #this value was empty
max_power = max_power.astype('float64')

In [9]:
df['max_power'] = max_power
df.rename(columns = {'max_power' : 'max_power(bhp)'}, inplace = True)

In [10]:
#Check for NaN
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8127 entries, 0 to 8127
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   brand           8127 non-null   object 
 1   year            8127 non-null   int64  
 2   selling_price   8127 non-null   int64  
 3   km_driven       8127 non-null   int64  
 4   fuel            8127 non-null   object 
 5   seller_type     8127 non-null   object 
 6   transmission    8127 non-null   object 
 7   owner           8127 non-null   object 
 8   mileage(kmpl)   7907 non-null   float64
 9   engine(cc)      7907 non-null   float64
 10  max_power(bhp)  7912 non-null   float64
 11  seats           7907 non-null   float64
dtypes: float64(4), int64(3), object(5)
memory usage: 860.7+ KB


In [11]:
#Replacing NaN values by the mean grouped by brand, fuel, transmission, seller_type, and owner
df['mileage(kmpl)'].fillna(df.groupby(['brand', 'fuel', 'transmission', 'seller_type', 'owner'])['mileage(kmpl)'].transform('mean'), inplace = True)
df['engine(cc)'].fillna(df.groupby(['brand', 'fuel', 'transmission', 'seller_type', 'owner'])['engine(cc)'].transform('mean'), inplace = True)
df['max_power(bhp)'].fillna(df.groupby(['brand', 'fuel', 'transmission', 'seller_type', 'owner'])['max_power(bhp)'].transform('mean'), inplace = True)
df['seats'].fillna(df.groupby(['brand', 'fuel', 'transmission', 'seller_type', 'owner'])['seats'].transform('median'), inplace = True)

In [12]:
#recheck for NaN
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8127 entries, 0 to 8127
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   brand           8127 non-null   object 
 1   year            8127 non-null   int64  
 2   selling_price   8127 non-null   int64  
 3   km_driven       8127 non-null   int64  
 4   fuel            8127 non-null   object 
 5   seller_type     8127 non-null   object 
 6   transmission    8127 non-null   object 
 7   owner           8127 non-null   object 
 8   mileage(kmpl)   8125 non-null   float64
 9   engine(cc)      8125 non-null   float64
 10  max_power(bhp)  8125 non-null   float64
 11  seats           8125 non-null   float64
dtypes: float64(4), int64(3), object(5)
memory usage: 860.7+ KB


In [13]:
# Finalizing replacing NaN values some where very specific and did not have a groupby values to generate mean
avg = df.iloc[:, [8,9,10]].mean(axis = 0)
median = df.iloc[:, 11].median(axis = 0)
df.iloc[:, [8,9,10]] = df.iloc[:, [8,9,10]].fillna(avg)
df.iloc[:, 11] = df.iloc[:, 11].fillna(median)

In [14]:
#rearrage columns, set price to last before getting training and test set
new_cols = ['brand', 'year', 'km_driven', 'fuel', 'seller_type', 'transmission', 'owner', 'mileage(kmpl)', 'engine(cc)', 'max_power(bhp)', 'seats', 'selling_price']
df = df[new_cols]
df.head()

,brand,year,km_driven,fuel,seller_type,transmission,owner,mileage(kmpl),engine(cc),max_power(bhp),seats,selling_price
0,Maruti,2014,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74.00,5.0,450000
1,Skoda,2014,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,5.0,370000
2,Honda,2006,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78.00,5.0,158000
3,Hyundai,2010,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90.00,5.0,225000
4,Maruti,2007,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.20,5.0,130000


## Save data to a new file 

In [15]:
df.to_csv('used_car_clean.csv', index = False)